As Motion Prediction has temporal data, I tried to used LSTM based seq 2 seq generation (Encoder-Decoder Model approach) atleast inspired.
Some part of code (validation) is borrowed from @ Marco B notebook . Model itself is quite basic and this can get to a score of 400 on leaderboard.

I hope this trigger some more work using LSTM and more sharing to come.

In [ ]:
!pip install l5kit
!pip install -U PyYAML

In [ ]:
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)

In [ ]:
 import numpy as np

import os
import torch
torch.manual_seed(0)

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18,resnet50,resnet101
from tqdm import tqdm
from typing import Dict
from torch import functional as F

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

In [ ]:
# set env variable for data
os.environ['L5KIT_DATA_FOLDER'] = '/content/drive/Shared drives/social-LSTM/l5kit/'
dm = LocalDataManager(None)
VALIDATION = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#cfg = load_config_data("/content/drive/My Drive/l5kit/examples/agent_motion_prediction/agent_motion_config.yaml")

cfg = load_config_data('/content/drive/Shared drives/social-LSTM/l5kit/examples/agent_motion_prediction/agent_motion_config.yaml')
cfg

In [ ]:
# ===== INIT DATASET
train_cfg = cfg["train_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Train dataset/dataloader
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset,
                              shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"],
                              num_workers=4)#train_cfg["num_workers"])


tr_it = iter(train_dataloader)

data = next(tr_it)

batch_size, step_size, fea_size = data['history_positions'].shape
# step_size = 20



#Need to modify
input_dim = fea_size
hidden_dim = fea_size
output_dim = fea_size
predic_time_seq_len = 50
encoder_h_dim = hidden_dim
decoder_h_dim = hidden_dim
print(train_dataset)
print(step_size)

In [ ]:
print(fea_size)

In [ ]:
print()

In [ ]:
# ===== INIT  VAL DATASET
val_cfg = cfg["val_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Train dataset/dataloader
val_zarr = ChunkedDataset(dm.require(val_cfg["key"])).open()
val_dataset = AgentDataset(cfg, val_zarr, rasterizer)
val_dataloader = DataLoader(val_dataset,
                              shuffle=val_cfg["shuffle"],
                              batch_size=val_cfg["batch_size"],
                              num_workers=train_cfg["num_workers"])

print(val_dataset)

In [ ]:
import numpy as np

train = 5
pred = 1

def data_handle_for_lstm(dir, train_seq_len= train, pred_seq_len=pred, frame_lenth_cap=train + pred):
#     '''
#     This function returns train sequences and prediction sequences for LSTM and RNN
#     :param dir: directory where the dataset is present
#     :param train_seq_len: length of the train sequence
#     :param pred_seq_len: length of the prediction sequence
#     :param frame_lenth_cap: minimum length of time presence the agent must have
#     :return: train sequences and prediction sequences
#     '''

     data = np.load(dir)
     d_IDs = np.unique(data[:, 4]).astype(int)

     train_sequence = []
     pred_sequence = []
     sz = len(d_IDs)

     mx = 0

     agent_IDs = np.unique(data[:,1]).astype(int)

     total_frames = int(np.amax(data[:, 0]))

     # traversing through each agent from all agents corresponding to that dataset
     for agent_id in agent_IDs:

         # obtaining the trajectory of that individual agent
         one_agent_traj = data[np.where(data[:, 1] == agent_id)]  # each_agent_id

         # selecting the agent only if it is visible in at least 'frame_length_cap' number of frames
         mx = max(mx, len(one_agent_traj[:, 0]))

         if len(one_agent_traj[:, 0]) >= frame_lenth_cap:

             # initializing an index range to obtain the train sequences and pred sequences
             index_range = len(one_agent_traj[:, 0]) - train_seq_len - pred_seq_len

             # obtaining the sequences
             for idx in range(1, index_range + 2):
                 train_sequence_dict = {}
                 pred_sequence_dict = {}

                 train_sequence_dict['agent_ID'] = agent_id

                 # train_sequence_dict['sequence'] = one_agent_traj[idx:idx+train_seq_len,2:4]
                 #Get its (x, y) under the time range
                 train_sequence_dict['sequence'] = one_agent_traj[idx - 1:idx - 1 + train_seq_len, 2:4]

                 pred_sequence_dict['agent_ID'] = agent_id

                 # pred_sequence_dict['sequence'] = one_agent_traj[idx+train_seq_len:idx+train_seq_len+pred_seq_len,2:4]
                 #This is for test the accuracy
                 pred_sequence_dict['sequence'] = one_agent_traj[
                                                  idx - 1 + train_seq_len:idx - 1 + train_seq_len + pred_seq_len,
                                                  2:4]

                 train_sequence.append(train_sequence_dict)
                 pred_sequence.append(pred_sequence_dict)

     return train_sequence, pred_sequence

# **Pre-Process data**

---


# **Encoder LSTM**

---



In [ ]:
class Encoder ( nn.Module ):
    def __init__ ( self , input_size , cell_size , hidden_size ):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        """
        super ( Encoder , self ).__init__ ()

        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear ( input_size + hidden_size , hidden_size )
        self.il = nn.Linear ( input_size + hidden_size , hidden_size )
        self.ol = nn.Linear ( input_size + hidden_size , hidden_size )
        self.Cl = nn.Linear ( input_size + hidden_size , hidden_size )

    #Here x1, y1 are the nodes with (x,y)
    def computeDist ( self , x1 , y1 ):
        return np.abs ( x1 - y1 )

    #This is what we can use for social LSTM later (haven't defined)
    def computeKNN ( self , curr_dict , ID , k ):
        import heapq
        from operator import itemgetter

        ID_x = curr_dict[ ID ]
        dists = {}
        for j in range ( len ( curr_dict ) ):
            if j != ID:
                dists[ j ] = self.computeDist ( ID_x , curr_dict[ j ] )
        KNN_IDs = dict ( heapq.nsmallest ( k , dists.items () , key=itemgetter ( 1 ) ) )
        neighbors = list ( KNN_IDs.keys () )

        return neighbors
        # return [1,2,3]

    def compute_A ( self , xt ):
        # return Variable(torch.Tensor(np.ones([xt.shape[0],xt.shape[0]])).cuda())
        xt = xt.cpu ().detach ().numpy ()
        A = np.zeros ( [ xt.shape[ 0 ] , xt.shape[ 0 ] ] )
        for i in range ( len ( xt ) ):
            #if xt[ i ] is not None:
            if xt[i][0] and xt[i][1] :
                neighbors = self.computeKNN ( xt , i , 4 )
                for neighbor in neighbors:
                    # if neighbor in labels:
                    # if idx < labels.index ( neighbor ):
                    A[ i ][ neighbor ] = 1
        return Variable ( torch.Tensor ( A ).cuda () )


    def forward ( self , input , Hidden_State , Cell_State ):

        combined = torch.cat ( (input , Hidden_State) , 1 )
        f = torch.sigmoid ( self.fl ( combined ) )
        i = torch.sigmoid ( self.il ( combined ) )
        o = torch.sigmoid ( self.ol ( combined ) )
        g = torch.tanh ( self.Cl ( combined ) )
        Cell_State = f * Cell_State + i * g
        Hidden_State = o * torch.tanh ( Cell_State )

        return Hidden_State , Cell_State

    def loop ( self , inputs ):
        batch_size = inputs.size ( 0 )
        time_step = inputs.size ( 1 )
        Hidden_State , Cell_State = self.initHidden ( batch_size )
        for i in range ( time_step ):
            Hidden_State , Cell_State = self.forward(torch.squeeze(inputs[:, i:i+1,:]), Hidden_State, Cell_State)
        return Hidden_State , Cell_State

    def initHidden ( self , batch_size ):
        Hidden_State = Variable ( torch.zeros ( batch_size , self.hidden_size ).to ( device ) )
        Cell_State = Variable ( torch.zeros ( batch_size , self.hidden_size ).to ( device ) )
        return Hidden_State , Cell_State



# **Revised Encoder part(Social-Pooling Layer)**

---

Here I want my input will be 1) hidden_state output for encoder, 2) time_neighbor matrix (identity matrix) 3) coordination  to create the model

Here input at following part is dataframe for whole agents: [agent 1, agent 2, ... agent_n]

I assume that you each time we only have one agent to feed so that here it is not the matrix, it should be vector.

In [ ]:
def make_mlp(dim_list, activation='relu', batch_norm=False, dropout=0):
    layers = []
    for dim_in, dim_out in zip(dim_list[:-1], dim_list[1:]):
        layers.append(nn.Linear(dim_in, dim_out))
        if batch_norm:
            layers.append(nn.BatchNorm1d(dim_out))
        if activation == 'relu':
            layers.append(nn.ReLU())
        elif activation == 'leakyrelu':
            layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(p=dropout))
    return nn.Sequential(*layers)


class SocialPooling(nn.Module):
    def __init__(
        self, h_dim, activation='relu', batch_norm=True, dropout=0.0,
        neighborhood_size=2.0, grid_size=8, pool_dim=None
    ):
        super(SocialPooling, self).__init__()
        self.h_dim = h_dim
        self.grid_size = grid_size
        self.neighborhood_size = neighborhood_size
        if pool_dim:
            mlp_pool_dims = [grid_size * grid_size * h_dim, pool_dim]
        else:
            mlp_pool_dims = [grid_size * grid_size * h_dim, h_dim]

        self.mlp_pool = make_mlp(
            mlp_pool_dims,
            activation=activation,
            batch_norm=batch_norm,
            dropout=dropout
        )

    def get_bounds(self, ped_pos):
        top_left_x = ped_pos[:, 0] - self.neighborhood_size / 2
        top_left_y = ped_pos[:, 1] + self.neighborhood_size / 2
        bottom_right_x = ped_pos[:, 0] + self.neighborhood_size / 2
        bottom_right_y = ped_pos[:, 1] - self.neighborhood_size / 2
        top_left = torch.stack([top_left_x, top_left_y], dim=1)
        bottom_right = torch.stack([bottom_right_x, bottom_right_y], dim=1)
        return top_left, bottom_right

    def get_grid_locations(self, top_left, other_pos):
        cell_x = torch.floor(
            ((other_pos[:, 0] - top_left[:, 0]) / self.neighborhood_size) *
            self.grid_size)
        cell_y = torch.floor(
            ((top_left[:, 1] - other_pos[:, 1]) / self.neighborhood_size) *
            self.grid_size)
        grid_pos = cell_x + cell_y * self.grid_size
        return grid_pos

    def repeat(self, tensor, num_reps):
        """
        Inputs:
        -tensor: 2D tensor of any shape
        -num_reps: Number of times to repeat each row
        Outpus:
        -repeat_tensor: Repeat each row such that: R1, R1, R2, R2
        """
        col_len = tensor.size(1)
        #print(col_len)
        
        tensor = tensor.unsqueeze(dim=1).repeat(1, num_reps, 1)

        #print('starrrrrrrrr')
        
        
        tensor = tensor.view(-1, col_len)
        
        #print(tensor.shape)

        return tensor

    def forward(self, h_states, seq_start_end, end_pos):
        """
        Inputs:
        - h_states: Tesnsor of shape (batch, h_dim)
        - seq_start_end: A list of tuples which delimit sequences within batch.
        - end_pos: Absolute end position of obs_traj (batch, 2)
        Output:
        - pool_h: Tensor of shape (batch, h_dim)
        """
        pool_h = []
        for _, x in enumerate(seq_start_end):
            
            start = x[0][0].item()
            end = x[0][1].item()
            num_ped = end - start
            grid_size = self.grid_size * self.grid_size

            curr_hidden = h_states.view(-1, self.h_dim)[start:end]

            #print(curr_hidden.shape)


            curr_hidden_repeat = curr_hidden.repeat(num_ped, 1)

            #print("this is the output for curr_hidden_repeat")
            #print(curr_hidden_repeat.shape)

            curr_hidden_repeat = curr_hidden_repeat.to(device = device)


            curr_end_pos = end_pos[start:end]

            curr_pool_h_size = (num_ped * grid_size) + 1
            curr_pool_h = curr_hidden.new_zeros((curr_pool_h_size, self.h_dim))
            
            curr_pool_h = curr_pool_h.to(device = device)
            
            # curr_end_pos = curr_end_pos.data
            top_left, bottom_right = self.get_bounds(curr_end_pos)

            # Repeat position -> P1, P2, P1, P2
            
            #print('start curr_end_pos')
            
            
            curr_end_pos = curr_end_pos.repeat(num_ped, 1)

            #print(curr_end_pos.shape)

            


            # Repeat bounds -> B1, B1, B2, B2
            top_left = self.repeat(top_left, num_ped)
            bottom_right = self.repeat(bottom_right, num_ped)

            grid_pos = self.get_grid_locations(
                    top_left, curr_end_pos).type_as(seq_start_end)
            # Make all positions to exclude as non-zero
            # Find which peds to exclude
            x_bound = ((curr_end_pos[:, 0] >= bottom_right[:, 0]) +
                       (curr_end_pos[:, 0] <= top_left[:, 0]))
            y_bound = ((curr_end_pos[:, 1] >= top_left[:, 1]) +
                       (curr_end_pos[:, 1] <= bottom_right[:, 1]))

            within_bound = x_bound + y_bound
            within_bound[0::num_ped + 1] = 1  # Don't include the ped itself
            within_bound = within_bound.view(-1)

            # This is a tricky way to get scatter add to work. Helps me avoid a
            # for loop. Offset everything by 1. Use the initial 0 position to
            # dump all uncessary adds.
            grid_pos += 1
            total_grid_size = self.grid_size * self.grid_size
            offset = torch.arange(
                0, total_grid_size * num_ped, total_grid_size
            ).type_as(seq_start_end)

            offset = self.repeat(offset.view(-1, 1), num_ped).view(-1)

            #print(offset.shape)
            #print(grid_pos.shape)


            #grid_pos += offset
            grid_pos[within_bound != 0] = 0

            curr_hidden_repeat = curr_hidden_repeat.to(device)
            grid_pos = grid_pos.to(device)
            curr_pool_h = curr_pool_h.to(device)

            grid_pos = grid_pos.view(-1, 1).expand_as(curr_hidden_repeat)

            curr_pool_h = curr_pool_h.scatter_add(0, grid_pos,
                                                  curr_hidden_repeat)
            curr_pool_h = curr_pool_h[1:]
            curr_pool_h = curr_pool_h.to(device = device)
            pool_h.append(curr_pool_h.view(num_ped, -1))

        pool_h = torch.cat(pool_h, dim=0)
        pool_h = self.mlp_pool(pool_h)

        
        pool_h = pool_h.narrow(0, 0, h_states.shape[0])

        #print('this is the final pool_h size')
        #print(pool_h.shape)


        pool_h = pool_h.to(device)
        return pool_h

# **Decoder LSTM**

---



In [ ]:
class Decoder(nn.Module):

    # def __init__(self, cfg):
    def __init__(self, input_size, cell_size, hidden_size, batchsize, timestep):
        super(Decoder, self).__init__()
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.batch_size = batchsize
        self.time_step = timestep
        self.num_mog_params = 5
        self.sampled_point_size = 2
        #self.stream = stream
        self.stream_specific_param = self.num_mog_params
        self.fl = nn.Linear(self.stream_specific_param + hidden_size, hidden_size)
        self.il = nn.Linear(self.stream_specific_param + hidden_size, hidden_size)
        self.ol = nn.Linear(self.stream_specific_param + hidden_size, hidden_size)
        self.Cl = nn.Linear(self.stream_specific_param + hidden_size, hidden_size)
        self.linear1 = nn.Linear(cell_size, self.stream_specific_param)
        # self.one_lstm = nn.LSTMCell
        # self.linear2 = nn.Linear ( self.sampled_point_size ,  hidden_size )

        #print("Hidden_size")
        #print(self.hidden_size)
        #print("stream_spefic_param")
        #print(self.stream_specific_param)



    def forward(self, input, Hidden_State, Cell_State):

        #print('input~')
        #print(input.shape)

        combined = torch.cat((input, Hidden_State), 1)

        #print("Input shape")
        #print(input.shape)
        #print("Hidden_State shape")
        #print(Hidden_State.shape)
        #print("Combined shape")
        #print(combined.shape)

        

        f = torch.sigmoid(self.fl(combined))
        i = torch.sigmoid(self.il(combined))
        o = torch.sigmoid(self.ol(combined))
        g = torch.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * g
        Hidden_State = o * torch.tanh(Cell_State)

        return Hidden_State, Cell_State

    def loop(self, hidden_vec_from_encoder):
        batch_size = self.batch_size
        time_step = self.time_step

        init_hidden_output = self.initHidden()

        Cell_State, out = init_hidden_output[0], init_hidden_output[1]

        mu1_all , mu2_all , sigma1_all , sigma2_all , rho_all = self.initMogParams()

        for i in range(time_step):
            if i == 0:
                Hidden_State = hidden_vec_from_encoder
            Hidden_State, Cell_State = self.forward(out, Hidden_State, Cell_State)

            mog_params = self.linear1(Hidden_State)

            out = mog_params

            mu_1 , mu_2 , log_sigma_1 , log_sigma_2 , pre_rho = mog_params.chunk ( 6 , dim=-1 )
            rho = torch.tanh ( pre_rho )
            log_sigma_1 = torch.exp(log_sigma_1)
            log_sigma_2 = torch.exp(log_sigma_2)
            mu1_all[:,i,:] = mu_1
            mu2_all[:,i,:] = mu_2
            sigma1_all[:,i,:] = log_sigma_1
            sigma2_all[:,i,:] = log_sigma_2
            rho_all[:,i,:] = rho

        Stream_output  = torch.cat((mu1_all,mu2_all), dim=2)

        return Stream_output, Cell_State


    def initHidden(self):
        out = torch.randn(self.batch_size, self.num_mog_params, device=device)
        
        return torch.randn(self.batch_size, self.hidden_size, device=device), out
    

    #Here i want to store (x,y) for the future predictions
    def initMogParams(self):
        mu1_all = torch.randn(self.batch_size, self.time_step, 1, device=device)
        mu2_all = torch.randn(self.batch_size, self.time_step, 1, device=device)
        sigma1_all = torch.randn(self.batch_size, self.time_step, 1, device=device)
        sigma2_all = torch.randn(self.batch_size, self.time_step, 1, device=device)
        rho_all = torch.randn(self.batch_size, self.time_step, 1, device=device)

        return mu1_all, mu2_all, sigma1_all, sigma2_all, rho_all    


# **Original LSTM**

---



In [ ]:
# ==== INIT MODEL
learning_rate = 1e-3

encoder_stream = Encoder(input_dim, hidden_dim, output_dim).to(device)

decoder_stream = Decoder(input_dim, hidden_dim, output_dim, batch_size, predic_time_seq_len).to(device)


encoder_stream_optimizer = optim.RMSprop(encoder_stream.parameters(), lr=learning_rate)
decoder_stream_optimizer = optim.RMSprop(decoder_stream.parameters(), lr=learning_rate)

#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=7000,gamma=0.1)
criterion = nn.MSELoss()

In [ ]:
def train_stream(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer):
    target_length = target_tensor.size(0)

    Hidden_State, _ = encoder.loop(input_tensor)


    #print(Hidden_State.shape)

    stream_out, _ = decoder.loop(Hidden_State)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    #print("This is the beginning!!!!")
    #print(target_tensor.shape)

    #print("This is the output for the stream we want to predict")
    #print(stream_out.shape)

    loss = criterion(target_tensor, stream_out)

    #loss = loss if loss > 0 else -1 * loss

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss

In [ ]:
import pdb
from torch.autograd import Variable

# ==== TRAIN LOOP
tr_it = iter(train_dataloader)
vl_it = iter(val_dataloader)

progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []
losses_mean_train = []
losses_val = []
losses_mean_val = []

for itr in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    
    #You need to train two parts
    encoder_stream.train()
    
    decoder_stream.train()
    torch.set_grad_enabled(True)

    # Forward pass
    history_positions = data['history_positions'].to(device)
    history_availabilities = data['history_availabilities'].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets_position = data["target_positions"].to(device)
    
    # data_handle_for_lstm(dir, train_seq_len= train, pred_seq_len=pred, frame_lenth_cap=train + pred)

    # not all the output steps are valid, but we can filter them out from the loss using availabilities

    #print(history_positions.shape)
    #print(targets_position.shape)

    #print(history_positions)
    #print(targets_position)


    loss = train_stream(history_positions, targets_position, encoder_stream, decoder_stream, encoder_stream_optimizer, decoder_stream_optimizer)
    # loss = train_stream(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer)
    loss = loss.mean()

    print(loss.item())
    
    losses_train.append(loss.item())
    losses_mean_train.append(np.mean(losses_train))

In [ ]:
print(losses_mean_train)

# **Create a Class about all social parts**

---



In [ ]:
class SocialLSTM(nn.Module):
  def __init__(self, input_size, output_size, predic_tim_seq_len, encoder_h_dim, decoder_h_dim, mlp_dim = 1024, 
                dropout = 0.0, activation = 'relu', batch_norm = True, neighborhood_size = 2.0, grid_size = 8, bottleneck_dim=1024):
    super(SocialLSTM, self).__init__()

    #self.input_size = input_size
    #self.obs_len = obs_len
    self.pred_len = predic_time_seq_len
    #self.seq_len = obs_len + pred_len
    self.mlp_dim = mlp_dim
    self.seq_start_end = seq_start_end
    
    self.decoder_input = input_size

    self.encoder = Encoder(input_size, encoder_h_dim, output_size)
    self.decoder = Decoder(input_size, decoder_h_dim, output_size, batch_size, predic_time_seq_len)
    self.pool_net = SocialPooling(h_dim = encoder_h_dim, activation  = activation, batch_norm = batch_norm, dropout = dropout, neighborhood_size=neighborhood_size,grid_size = grid_size)


    

    #Decoder hidden:
    #input_dim = encoder_h_dim + bottleneck_dim

    mlp_decoder_context_dims = [input_dim, mlp_dim, decoder_h_dim]

    self.mlp_decoder_context = make_mlp(
                mlp_decoder_context_dims,
                activation=activation,
                batch_norm=batch_norm,
                dropout=dropout
            )




  def forward(self, input_tensor, obs_traj, seq_start_end):
    #Here I want to know the last one end_pos
    #I assume that obs_traj: Tensor of shape (obs_len, batch, 2)


    encoder_hidden_state, _  = self.encoder.loop(input_tensor)

    end_pos = obs_traj[:,-1,:]

    pool_h = self.pool_net(encoder_hidden_state, seq_start_end, end_pos)

    #print(pool_h.shape)
    #print(encoder_hidden_state.shape)
    
    x = pool_h + encoder_hidden_state
    #print(x.shape)


    mlp_decoder_context_input = torch.cat([encoder_hidden_state.view(-1, encoder_h_dim), pool_h], dim=1)

    #decoder_hidden = torch.unsqueeze(mlp_decoder_context_input, 0)

    #Original one
    #decoder_hidden = mlp_decoder_context_input

    decoder_hidden = x

    #print('decoder_hidden')
    #print(decoder_hidden.shape)
    #print('mlp')
    #print(mlp_decoder_context_input.shape)

    stream_out, _ = self.decoder.loop(decoder_hidden)



    return stream_out





# **Prepare the data for the start and end**

---

Here I want to return a dataframe 


In [ ]:
def parse_scene(scene):
    scene_dict = {
            "frame_index_interval_start": scene[0][0],
            "frame_index_interval_end": scene[0][1],
            "host":  scene[1],
            "start_time": scene[2],
            "end_time": scene[3]
        }
    return scene_dict

In [ ]:
DATA_ROOT = Path("/content/drive")

class BaseParser:
    """
    A robust and fast interface to load l5kit data into  Pandas dataframes.

    Parameters
    ----------
    chunk_size: int, default=1000
        How many items do you want in a single slice. The larger the better;
        as long as you have enough memory. Nevertheless, chunk sizes above `10_000` won't lead to
        significant speed gain as the original zarr files was chunked at 10_000.

    max_chunks: int, default=10
        How many chunks do you want to read from memory.

    root:
        Zarr data root path

    zarr_path:
        relative path or key to the data.
    """
    
    field = "scenes"
    dtypes = {}
    
    def __init__(self, start=0, end=None, chunk_size=1000, max_chunks=10, root=DATA_ROOT, zarr_path="Shared drives/social-LSTM/l5kit/scenes/sample.zarr"):
        
        self.start = start
        self.end = end
        self.chunk_size = chunk_size
        self.max_chunks = max_chunks
        

        self.root = Path(root)
        assert self.root.exists(), "There is nothing at {}!".format(self.root)
        self.zarr_path = Path(zarr_path)
        
     
    def parse(self):
        raise NotImplementedError
        
    def to_pandas(self, start=0, end=None, chunk_size=None, max_chunks=None):
        start = start or self.start
        end = end or self.end
        chunk_size = chunk_size or self.chunk_size
        max_chunks = max_chunks or self.max_chunks
        
        if not chunk_size or  not max_chunks: # One shot load, suitable for small zarr files
            df = zarr.load(self.root.joinpath(self.zarr_path).as_posix()).get(self.field)
            df = df[start:end]
            df = map(self.parse, df) 
        else: # Chunked load, suitable for large zarr files
            df = []
            with zarr.open(self.root.joinpath(self.zarr_path).as_posix(), "r") as zf:
                end = start+max_chunks*chunk_size if end is None else min(end, start+max_chunks*chunk_size)
                for i_start in range(start, end, chunk_size ):
                    items = zf[self.field][i_start: min(i_start + chunk_size,end)]
                    items = map(self.parse, items)
                    df.append(items)
            df = it.chain(*df)
            
        df = pd.DataFrame.from_records(df)
        for col, col_dtype in self.dtypes.items():
            df[col] = df[col].astype(col_dtype, copy=False)
        return df

In [ ]:
class SceneParser(BaseParser):
    field = "scenes"
    
    @staticmethod
    def parse(scene):
        scene_dict = {
            "frame_index_interval_start": scene[0][0],
            "frame_index_interval_end": scene[0][1],
            "host":  scene[1],
            "start_time": scene[2],
            "end_time": scene[3]
        }
        return scene_dict

In [ ]:
def parse_frame(frame):
    frame_dict = {
        'timestamp': frame[0],
        'agent_index_interval_start': frame[1][0],
        'agent_index_interval_start': frame[1][1],
        'traffic_light_faces_index_interval_start': frame[2][0],
        'traffic_light_faces_index_interval_end': frame[2][1],
        'ego_translation_x': frame[3][0],
        'ego_translation_y': frame[3][1],
        'ego_translation_z': frame[3][2],
        'ego_rotation_xx': frame[4][0][0],
        'ego_rotation_xy': frame[4][0][1],
        'ego_rotation_xz': frame[4][0][2],
        'ego_rotation_yx': frame[4][1][0],
        'ego_rotation_yy': frame[4][1][1],
        'ego_rotation_yz': frame[4][1][2],
        'ego_rotation_zx': frame[4][2][0],
        'ego_rotation_zy': frame[4][2][1],
        'ego_rotation_zz': frame[4][2][2],
        
    }
    return frame_dict

In [ ]:
class FrameParser(BaseParser):
    field = "frames"
    
    @staticmethod
    def parse(frame):
        frame_dict = {
            'timestamp': frame[0],
            'agent_index_interval_start': frame[1][0],
            'agent_index_interval_end': frame[1][1],
            'traffic_light_faces_index_interval_start': frame[2][0],
            'traffic_light_faces_index_interval_end': frame[2][1],
            'ego_translation_x': frame[3][0],
            'ego_translation_y': frame[3][1],
            'ego_translation_z': frame[3][2],
            'ego_rotation_xx': frame[4][0][0],
            'ego_rotation_xy': frame[4][0][1],
            'ego_rotation_xz': frame[4][0][2],
            'ego_rotation_yx': frame[4][1][0],
            'ego_rotation_yy': frame[4][1][1],
            'ego_rotation_yz': frame[4][1][2],
            'ego_rotation_zx': frame[4][2][0],
            'ego_rotation_zy': frame[4][2][1],
            'ego_rotation_zz': frame[4][2][2],

        }
        return frame_dict

    def to_pandas(self, start=0, end=None, chunk_size=None, max_chunks=None, scene=None):
        if scene is not None:
            start = scene.frame_index_interval_start
            end = scene.frame_index_interval_end
        
        df = super().to_pandas(start=start, end=end, chunk_size=chunk_size, max_chunks=max_chunks)
        return df

# **Input the data to generate the data**

---




In [ ]:
import zarr
import pandas as pd, numpy as np

import itertools as it # I will be using the `itertools.chain` function
from pathlib import Path # for better file/path operations management

DATA_ROOT = Path("/content/drive")

zl5 = zarr.open(DATA_ROOT.joinpath("Shared drives/social-LSTM/l5kit/scenes/sample.zarr").as_posix(), mode="r")

sp = SceneParser(chunk_size=None, max_chunks=None, zarr_path="/content/drive/Shared drives/social-LSTM/l5kit/scenes/sample.zarr")

scenes = sp.to_pandas()

row_scene, col_scene = scenes.shape

all_information = pd.DataFrame()


for i in range(row_scene):
  scene = scenes.iloc[i]
  scene_frames = zl5.frames[scene.frame_index_interval_start:scene.frame_index_interval_end]
  
  fp = FrameParser()
  frames = fp.to_pandas(scene = scene)

  frames_needed = frames.iloc[:,0:3]

  if i == 0:
    all_information = frames_needed
  else:
    all_information = all_information.append(frames_needed, ignore_index=True)

all_information['num_agent'] = all_information['agent_index_interval_end'] - all_information['agent_index_interval_start']

#xx = all_information.loc[all_information['timestamp'] ==1572643684801892606]

#print(xx.iloc[0]['num_agent'])



In [ ]:
zl5.scenes.info

In [ ]:
print(scenes.head())

# **Overall revised code**

---



In [ ]:
# ==== INIT MODEL
learning_rate = 1e-3

seq_start_end = torch.LongTensor()

slstm = SocialLSTM( input_size = input_dim, output_size= output_dim, predic_tim_seq_len= predic_time_seq_len, encoder_h_dim= hidden_dim, decoder_h_dim=decoder_h_dim).to(device)

slstm_optimizer = optim.RMSprop(slstm.parameters(), lr = 0.0001)

#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=7000,gamma=0.1)
criterion = nn.MSELoss()

In [ ]:
f = open("Social_LSTM_avg_res.csv", "r")
print(f.read()) 

In [ ]:
import pdb
from torch.autograd import Variable

# ==== TRAIN LOOP
tr_it = iter(train_dataloader)
vl_it = iter(val_dataloader)

progress_bar = tqdm(range(10000), position=0)#cfg["train_params"]["max_num_steps"]))
losses_train = []
losses_mean_train = []
losses_val = []
losses_mean_val = []

for i in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    
    #You need to train two parts
    slstm.train()
    torch.set_grad_enabled(True)


    #print(data)

    # Forward pass
    history_positions = data['history_positions'].to(device)
    history_availabilities = data['history_availabilities'].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets_position = data["target_positions"].to(device)
    centroid = data['centroid'].to(device)

    seq_start_end = list()

    
    for xx in (data['timestamp']):
      #print(xx.item())
      specific_row_dat = all_information.loc[all_information['timestamp'] ==xx.item()]
      cum_start_idx = [0] + np.cumsum(specific_row_dat.iloc[0]['num_agent']).tolist()
      
      seq_start_end.append([(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])])
    
    
    seq_start_end = torch.LongTensor(seq_start_end)
      
    #print(seq_start_end)

 #   for _, x in enumerate(seq_start_end):
#      print('sssss')
#      print(x)
      #print(x[0])
      #print(x[0][0])
#      start = x[0][0].item()
#      print(start)
#      end = x[0][1].item()
#      print(end)
      #start = x.item()
      #end = end.item()
      #print(start)
      #print(end)
      #start = x[0]
      #end = x[1]
      #print(start)
      #print(end)
 #     print(x[0])
 #     print(x[0][0])
 #     print(x[0][1])
 #     print(type(x))



    outputs = slstm(history_positions, history_positions, seq_start_end)

    loss = criterion(outputs, targets_position)

    loss = loss * target_availabilities
    loss = loss.mean()

    #Backward pass
    slstm_optimizer.zero_grad()
    loss.backward()
    slstm_optimizer.step()

    losses_train.append(loss.item())
    losses_mean_train.append(np.mean(losses_train))

    if (i % 250) == 250 - 1:
      with open('Social_LSTM_res2.csv','a') as fd:
          for loss in losses_train:
              fd.write(f"{i},{loss}\n")
      avg_loss = np.mean(losses_train)
      losses_train = []
      with open('Social_LSTM_avg_res2.csv','a') as fd:
          fd.write(f"{i},{avg_loss}\n")
      progress_bar.set_description(f"loss: {loss} loss(avg): {avg_loss}")

    # print(loss.item())

    
    

In [ ]:
nn# ===== INIT DATASET
test_cfg = cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"/content/drive/Shared drives/social-LSTM/l5kit/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataloader)

In [ ]:
model.eval()

future_coords_offsets_pd = []
timestamps = []
agent_ids = []

with torch.no_grad():
    dataiter = tqdm(test_dataloader)
    
    for data in dataiter:

        history_positions = data['history_positions'].to(device)

        outputs = model(history_positions)
        
        future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())


In [ ]:
write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))